### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = 4096,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!


==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L40S. Num GPUs = 1. Max memory: 44.418 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.6.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


<a name="Data"></a>
### Data Prep
We will use a custom dataset named `Interview_Data_6K.csv`. This dataset contains conversations with a mental health counselling assistant. Each entry has an `instruction` (acting as a system prompt), an `input` (the user's message), and an `output` (the assistant's response).

We need to convert this CSV data into a format suitable for training with `SFTTrainer`, specifically by applying the Qwen3 chat template.

In [4]:
from datasets import load_dataset, DatasetDict

# Load the custom CSV dataset
# The dataset has 'instruction', 'input', 'output' columns
full_interview_dataset = load_dataset("csv", data_files="./dataset/Interview_Data_6K.csv", split="train")

# Split the dataset into training and evaluation sets (e.g., 90% train, 10% eval)
# Ensure the dataset has more than one example for splitting
if len(full_interview_dataset) > 1:
    train_test_split = full_interview_dataset.train_test_split(test_size=0.1, seed=3407)
    interview_train_dataset = train_test_split['train']
    interview_eval_dataset = train_test_split['test']
    print(f"Training set size: {len(interview_train_dataset)}")
    print(f"Evaluation set size: {len(interview_eval_dataset)}")
else:
    # Handle cases with very small datasets
    interview_train_dataset = full_interview_dataset
    interview_eval_dataset = None
    print(f"Training set size: {len(interview_train_dataset)}")
    print("No evaluation set created due to small dataset size.")

# We will display the structure of the training dataset in the next cell.

Training set size: 5679
Evaluation set size: 631


Let's see the structure of our loaded training dataset:

In [5]:
interview_train_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5679
})

In [6]:
import pandas as pd # For pd.notna

# 'interview_train_dataset' and 'interview_eval_dataset' are from the previous cell
# The 'tokenizer' is globally defined in a prior cell (Cell 6, id="c1180838")

def convert_csv_to_chat_format(examples):
    """
    Converts a batch of examples from the CSV structure to a list of conversations.
    Each conversation is a list of dictionaries with "role" and "content".
    """
    all_conversations = []
    # 'examples' is a dictionary where keys are column names and values are lists of entries
    num_examples = len(examples['instruction'])
    
    for i in range(num_examples):
        instruction = examples['instruction'][i]
        input_text = examples['input'][i]
        output_text = examples['output'][i]

        # Ensure input_text is a string; handle None or NaN by treating as empty string if necessary
        input_text_str = str(input_text) if pd.notna(input_text) and str(input_text).strip() else ""

        current_conversation = []
        # System prompt from 'instruction'
        current_conversation.append({"role": "system", "content": instruction})
        
        # User message from 'input'
        # Based on the dataset, 'input' should always be present.
        # If input_text_str is empty, this will add a user message with empty content.
        # This is generally fine as the model should learn to handle it or it implies
        # the system prompt itself is the query.
        current_conversation.append({"role": "user", "content": input_text_str})
            
        # Assistant message from 'output'
        current_conversation.append({"role": "assistant", "content": output_text})
        
        all_conversations.append(current_conversation)
        
    return {"conversations": all_conversations}

def apply_template_to_conversations(examples):
    """
    Applies the tokenizer's chat template to a batch of conversations.
    Creates a 'text' field for SFTTrainer.
    """
    # tokenizer should be globally available
    return {
        "text": tokenizer.apply_chat_template(
            examples["conversations"], # This is a list of conversations
            tokenize=False,
            add_generation_prompt=False, # Crucial for training
        )
    }

# Process training data
# 1. Convert CSV structure to list of message dicts
train_dataset_with_conversations = interview_train_dataset.map(
    convert_csv_to_chat_format,
    batched=True,
    remove_columns=interview_train_dataset.column_names # Keep only 'conversations'
)
# 2. Apply chat template to create the 'text' field
final_train_dataset = train_dataset_with_conversations.map(
    apply_template_to_conversations,
    batched=True,
    remove_columns=["conversations"] # Keep only 'text'
)
# 3. Shuffle the training dataset
final_train_dataset = final_train_dataset.shuffle(seed=3407)

# Process evaluation data (if it exists)
final_eval_dataset = None
if interview_eval_dataset:
    eval_dataset_with_conversations = interview_eval_dataset.map(
        convert_csv_to_chat_format,
        batched=True,
        remove_columns=interview_eval_dataset.column_names
    )
    final_eval_dataset = eval_dataset_with_conversations.map(
        apply_template_to_conversations,
        batched=True,
        remove_columns=["conversations"]
    )
    # No need to shuffle eval_dataset

print("Sample of final formatted training data (after chat template):")
if len(final_train_dataset) > 0:
    print(final_train_dataset[0]['text'])
else:
    print("Training dataset is empty after processing.")

if final_eval_dataset and len(final_eval_dataset) > 0:
    print("\nSample of final formatted evaluation data (after chat template):")
    print(final_eval_dataset[0]['text'])
elif interview_eval_dataset: # If interview_eval_dataset existed but final_eval_dataset is empty
    print("\nEvaluation dataset is empty after processing.")

Sample of final formatted training data (after chat template):
<|im_start|>system
You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. 
The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. <|im_end|>
<|im_start|>user
I appreciate your understanding and the advice you've given me. I've been trying to focus on the positive memories, but I can't help feeling overwhelmed by the sadness and the sense of loss. I've tried reaching out to friends and family, but it feels like they don't truly understand what I'm going through. I've even considered seeking professional help, but I'm not sure if it's the right step for me.<|im_end|>
<|im_start|>assistant
<think>

</think>

I can see how difficult this time is for you, and it's essential to remember that everyone's grieving process is unique. It's common to feel overwhelmed by emotions and to struggle with finding ways to cope. 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [7]:
import wandb
wandb.init(project="distress-chatbot", name="base-model-training-2", config={
    "model": "Qwen/Qwen3-4B",
    # "max_steps": 20000,
    "learning_rate": 2e-5,
    "lambda_decay": 0.95,
})  # Allow resuming W&B run

from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_train_dataset, # Use the processed training data
    eval_dataset = final_eval_dataset,   # Use the processed evaluation data
    args = SFTConfig(
        dataset_text_field = "text", # Column containing the formatted text
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        # max_steps = 60, # Adjusted for quicker testing, was 30. Set to None or higher for full training.
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
        # evaluation_strategy = "steps" if final_eval_dataset else "no", # Enable evaluation if eval_dataset is present
        # eval_steps = 20, # Evaluate every N steps, adjust as needed
    ),
)

# If using evaluation, you might want to set evaluation_strategy and eval_steps in SFTConfig
if final_eval_dataset:
    trainer.args.evaluation_strategy = "steps"
    trainer.args.eval_steps = 20 # Or any other desired frequency
else:
    trainer.args.evaluation_strategy = "no"

wandb: Currently logged in as: hongtai91 (hongtai91-n-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Tracking run with wandb version 0.20.1


wandb: Run data is saved locally in /home/tai/nlp/wandb/run-20250614_161006-1sq711ff
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run base-model-training-2


wandb: ⭐️ View project at https://wandb.ai/hongtai91-n-a/distress-chatbot


wandb: 🚀 View run at https://wandb.ai/hongtai91-n-a/distress-chatbot/runs/1sq711ff


In [8]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L40S. Max memory = 44.418 GB.
4.373 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,679 | Num Epochs = 5 | Total steps = 1,775
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 264,241,152/4,000,000,000 (6.61% trained)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.335100
2,2.284300
3,2.553600
4,2.511900
5,2.393100
6,2.154900
7,1.970500
8,1.818900
9,1.665600
10,1.780500


In [10]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3619.4322 seconds used for training.
60.32 minutes used for training.
Peak reserved memory = 7.408 GB.
Peak reserved memory for training = 3.035 GB.
Peak reserved memory % of max memory = 16.678 %.
Peak reserved memory for training % of max memory = 6.833 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [13]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [1]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-15 07:21:59 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-15 07:21:59 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.7: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA L40S. Num GPUs = 1. Max memory: 44.418 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.5.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
# Save the model into merged version to continue doing the Reinforcement Learning
model.save_pretrained_merged("merged_based_model_sft", tokenizer, save_method="merged_16bit")  # Local saving

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 720.71 out of 1007.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 76.38it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [57]:
messages = [
    {"role": "system", "content": """You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description.The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. At the end of answer, add tag <evaluate>{"Active Listening" : score, "Empathy & Validation": score, "Safety & Trustworthiness" : score, "Open-mindedness & Non-judgment" : score, "Clarity & Encouragement" : score, "Boundaries & Ethical" : score, "Holistic Approach" : score, "Explaination for Scoring": explain} </evaluate> evaluate your consultant answer in 7 metrics and explain for that evaluation with score from 1 to 10 in json format, where 1 is the worst and 10 is the best and explain is clearly explain why has that score. \n\nConsultation Metrics:\n1. Active Listening: Responses should show careful consideration of the user's concerns, reflecting an understanding and capturing the essence of the issue. Avoid making assumptions or jumping to conclusions.\n2. Empathy & Validation: Responses should convey deep understanding and compassion, validating the user's feelings and emotions without being dismissive or minimizing their experiences.\n3. Safety & Trustworthiness: Prioritize user safety in responses, refraining from potentially harmful or insensitive language. Ensure that information provided is consistent and trustworthy.\n4. Open-mindedness & Non-judgment: Approach concerns without any inherent bias or judgment. Answers should be free from biases related to personal attributes and convey respect, demonstrating unconditional positive regard.\n5. Clarity & Encouragement: Provide clear, concise, and easily understandable answers. Where appropriate, motivate or highlight strengths, offering encouragement while maintaining a neutral stance.\n6. Boundaries & Ethical: It's vital to clarify the role of the response, emphasizing its informational nature. In complex scenarios, guiding users to seek human professional assistance is essential.\n7. Holistic Approach: Responses should be comprehensive, addressing concerns from various angles, be it emotional, cognitive, or situational. Consider the broader context, even if not explicitly detailed in the query."""},
    {"role" : "user", "content" : "Tôi đã gặp khó khăn trong việc tìm kiếm sự cân bằng giữa trách nhiệm công việc và vai trò làm mẹ đơn thân của một cậu con trai 12 tuổi. Tôi nhận thấy mình thường cảm thấy quá tải và lo lắng, và tôi đang cân nhắc việc thiết lập một lịch trình có cấu trúc hơn cho cả hai mẹ con. Tuy nhiên, tôi còn do dự trong việc đặt ra giới hạn nghiêm ngặt về thời gian sử dụng thiết bị điện tử, vì tôi nhận thấy rằng việc đặt giới hạn chặt chẽ đôi khi có thể dẫn đến tranh cãi và sự oán giận. Thay vào đó, tôi đang nghĩ đến các chiến lược linh hoạt hơn, như thưởng thêm thời gian sử dụng thiết bị hoặc các vật phẩm trong trò chơi khi con có hành vi tốt. Nhưng tôi không chắc liệu những phần thưởng này có còn là lựa chọn khả thi hay không, vì chúng có thể củng cố thói quen sử dụng thiết bị điện tử không lành mạnh."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 2048, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    repetition_penalty = 1.1,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Tôi hiểu rõ 

sự căng thẳng mà bạn đang trải qua khi phải đối mặt với những thử thách liên quan đến việc quản lý thời gian và cân bằng giữa công việc và cuộc sống gia đình. Việc đặt ra các giới hạn cứng rắn về thời gian sử dụng thiết bị điện tử có thể gây ra nhiều bất tiện và xung đột, đặc biệt là khi con cái trở nên phản kháng. Do đó, tôi khuyên bạn nên thử nghiệm các phương pháp linh hoạt hơn, chẳng hạn như việc thưởng thêm thời gian sử dụng thiết bị điện tử hoặc các vật phẩm trong trò chơi khi con có hành vi tích cực. Tuy nhiên, trước khi thực hiện điều này, hãy đảm bảo rằng bạn đã thảo luận kỹ lưỡng với con gái của bạn để đưa ra một thỏa thuận rõ ràng và dễ hiểu. Nếu bạn vẫn cảm thấy bối rối, đừng ngại liên hệ lại để được tư vấn thêm.

<evaluate>
{"Active Listening" : 9, "Empathy & Validation" : 8, "Safety & Trustworthiness" : 9, "Open-mindedness & Non-judgment" : 9, "Clarity & Encouragement" : 8, "Boundaries & Ethical" : 7, "Holistic Approach" : 9, "Explaination for Scoring": "The counselor lis

In [ ]:
from langdetect import detect

text = """Tôi hiểu rõ 
sự căng thẳng mà bạn đang trải qua khi phải đối mặt với những thử thách liên quan đến việc quản lý thời gian và cân bằng giữa công việc và cuộc sống gia đình. Việc đặt ra các giới hạn cứng rắn về thời gian sử dụng thiết bị điện tử có thể gây ra nhiều bất tiện và xung đột, đặc biệt là khi con cái trở nên phản kháng. Do đó, tôi khuyên bạn nên thử nghiệm các phương pháp linh hoạt hơn, chẳng hạn như việc thưởng thêm thời gian sử dụng thiết bị điện tử hoặc các vật phẩm trong trò chơi khi con có hành vi tích cực. Tuy nhiên, trước khi thực hiện điều này, hãy đảm bảo rằng bạn đã thảo luận kỹ lưỡng với con gái của bạn để đưa ra một thỏa thuận rõ ràng và dễ hiểu. Nếu bạn vẫn cảm thấy bối rối, đừng ngại liên hệ lại để được tư vấn thêm.
"""


In [60]:
from langdetect import detect

text = """Tôi hiểu rõ 
sự căng thẳng mà bạn đang trải qua khi phải đối mặt với những thử thách liên quan đến việc quản lý thời gian và cân bằng giữa công việc và cuộc sống gia đình. Việc đặt ra các giới hạn cứng rắn về thời gian sử dụng thiết bị điện tử có thể gây ra nhiều bất tiện và xung đột, đặc biệt là khi con cái trở nên phản kháng. Do đó, tôi khuyên bạn nên thử nghiệm các phương pháp linh hoạt hơn, chẳng hạn như việc thưởng thêm thời gian sử dụng thiết bị điện tử hoặc các vật phẩm trong trò chơi khi con có hành vi tích cực. Tuy nhiên, trước khi thực hiện điều này, hãy đảm bảo rằng bạn đã thảo luận kỹ lưỡng với con gái của bạn để đưa ra một thỏa thuận rõ ràng và dễ hiểu. Nếu bạn vẫn cảm thấy bối rối, đừng ngại liên hệ lại để được tư vấn thêm.
"""

print(detect(text))


vi


## Section 3: Reinforcement Learning to make model allow the structure

After getting the model training on dataset, I would like to continue finetuning the model on GRPO (Group Relative Policy Optimization) to strictly following the format of having self-evaluation like this:

<evaluate>
{"Active Listening" : 9, "Empathy & Validation" : 8, "Safety & Trustworthiness" : 9, "Open-mindedness & Non-judgment" : 9, "Clarity & Encouragement" : 8, "Boundaries & Ethical" : 7, "Holistic Approach" : 9, "Explaination for Scoring": "Explaination for score."}
</evaluate>

The library use in here is GRPO of UnSloth

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [15]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [16]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
